## Verisetinin Yüklenmesi

<span style="font-size:1.1em;">Colab'a Google drive'ı entegre ediyoruz. Kullanılacak olan veriseti Google Drive'da bulunmaktadır</span>

In [96]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
PREPROCESSED_DATASET_WITH_STEMMER = "gdrive/My Drive/mbti/preprocessed_dataset_with_stemming.csv"
PREPROCESSED_DATASET_WITHOUT_STEMMER = "gdrive/My Drive/mbti/preprocessed_dataset_no_stemming.csv"
PREPROCESSED_DATASET_ZEMBEREK = "gdrive/My Drive/mbti/preprocessed_dataset_zemberek.csv"
TRIMMED_DATASET = "gdrive/My Drive/mbti/trimmed_dataset.csv"
RAW_DATASET = "gdrive/My Drive/mbti/all_users_v2.csv"

<span style="font-size:1.1em;">Hangi veriseti kullanılarak işlem yapılacaksa yukardaki pathlerden biri seçilir ve parametre olarak verilir.</span>

In [0]:
import pandas as pd 
df = pd.read_csv(PREPROCESSED_DATASET_WITHOUT_STEMMER, sep = ';', header = 0)

In [99]:
df

,Unnamed: 0,user,entry,type,typeClass,I/E,S/N,T/F,J/P
0,0,19991991,ekşi itiraf döndüm dolaştım geldim kendime çat...,ENTJ,analysts,E,N,T,J
1,1,19991991,selda bağcan sesi dinleyebilmek büyük mutluluk...,ENTJ,analysts,E,N,T,J
2,2,19991991,eski sevgilisinin mutlu olmasını isteyen insan...,ENTJ,analysts,E,N,T,J
3,3,19991991,veda ederken a not bırakmak farklı acılar mu...,ENTJ,analysts,E,N,T,J
4,4,19991991,ingiliz aksanı aralar bayıldığım konuşan insan...,ENTJ,analysts,E,N,T,J
...,...,...,...,...,...,...,...,...,...
524835,643907,zaimoglu,zlatan ibrahimoviç türkiye katil,ESFJ,sentinels,E,S,F,J
524836,643908,zaimoglu,tarihteki büyük yalancılar şike operasyonu azi...,ESFJ,sentinels,E,S,F,J
524837,643909,zaimoglu,akp chp koalisyonu hayali koalisyondur,ESFJ,sentinels,E,S,F,J
524838,643910,zaimoglu,trabzonlu insana hamsi diyen fenerbahçe taraft...,ESFJ,sentinels,E,S,F,J


## Feature Extraction

<span style="font-size:1.1em;">TF-IDF özellik vektörünün çıkartılmasında kullanılacak değişken aşağıda belirlenmiş olan parametrelerle oluşturulur.</span>

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [0]:
tfidf_vectorizer = TfidfVectorizer(sublinear_tf=True, norm='l2', encoding='utf-8', ngram_range=(1, 2), max_features=5000, analyzer = 'word', token_pattern=r'\w{1,}')

In [0]:
import numpy as np

df['entry'] = df['entry'].apply(lambda x: np.str_(x)) # ValueError: np.nan is an invalid document seklinde bir hata verdigi icin bunu asmak adina yapildi.

<span style="font-size:1.1em;">Burada tf-idf vektörlerinin çıkartılmasında kullanılacak olan vocabulary oluşturulur.</span>

In [103]:
tfidf_vectorizer.fit(df['entry'])

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=5000,
                min_df=1, ngram_range=(1, 2), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=True, token_pattern='\\w{1,}', tokenizer=None,
                use_idf=True, vocabulary=None)

## Modelin Oluşturulması

<span style="font-size:1.1em;">Veriseti train ve test olmak üzere ikiye ayrılır. Test %20 ve train %80'ini oluşturacak şekilde tüm veriseti bölünür. random_state parametresi ile tekrardan bölündüğünde bir öncekiyle aynı train ve test veri setlerinin oluşturulması sağlanır.</span>

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
X_train, X_test, y_train, y_test = train_test_split(df['entry'], df['typeClass'], random_state = 42, test_size = 0.20)

Train ve test datasetlerinden tf-idf vektörleri çıkartılır


In [0]:
X_train_tfidf = tfidf_vectorizer.transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

Multinominal Naive Bayes modeli oluşturulur. Oluşturulan bu model verisetinde "type" olarak belirtilen "analysts", "diplomats", "sentimenls", "explorers" sınıflarından hangilerine ait olduğunu tahmin etmek için kullanılır

In [0]:
from sklearn.naive_bayes import MultinomialNB

In [0]:
clf = MultinomialNB().fit(X_train_tfidf, y_train)
test_typeClass = y_test.values

predictions = clf.predict(X_test_tfidf)

In [109]:
predictions

array(['analysts', 'diplomats', 'diplomats', ..., 'analysts', 'analysts',
       'analysts'], dtype='<U9')

<span style="font-size:1.1em">Yapılacak tahminlerle ilgili istatistiksel verileri tutmak için</span> ```predictions_result```<span style="font-size:1.1em"> adında bir değişken oluşturulur.</span>

<span style="font-size:1.1em">Bu değişkenin yapısı aşağıdaki gibidir.</span>

```json
{
    "predicted": {
        "analysts":  { "actual": {"analysts": 0, "diplomats": 0, "explorers": 0, "sentinels": 0} }
        "diplomats": { "actual": {"analysts": 0, "diplomats": 0, "explorers": 0, "sentinels": 0} }
        "explorers": { "actual": {"analysts": 0, "diplomats": 0, "explorers": 0, "sentinels": 0} }
        "sentinels": { "actual": {"analysts": 0, "diplomats": 0, "explorers": 0, "sentinels": 0} }
    }
}
```

* <span style="font-size:1.1em;">Yapılan tahminlerle ilgili verilere ulaşabilmek için</span>

    ```predictions_results['predicted']```


* <span style="font-size:1.1em;">Yapılan tahminin analyst ise:</span>

    ```predictions_results['predicted']['analysts']``` 


* <span style="font-size:1.1em;">Yapılan analyst tahmininin gerçek değerlerine erişmek için:</span>     

    ```predictions_results['predicted']['analysts']['actual']```  


* <span style="font-size:1.1em;">Test verisi, model tarafından analysts olarak tahmin edilmiştir ve bu verinin gerçek değeri de analysts'tir.</span>

    ```predictions_results['predicted']['analysts']['actual']['analysts']``` 

In [0]:
prediction_results = {'predicted': {}}  ## prediction_result['analysts'] means prediction is 'analysts'

prediction_results['predicted']['analysts']  = {'actual': {'analysts': 0, 'diplomats': 0, 'explorers': 0, 'sentinels': 0}}
prediction_results['predicted']['diplomats'] = {'actual': {'analysts': 0, 'diplomats': 0, 'explorers': 0, 'sentinels': 0}}
prediction_results['predicted']['explorers'] = {'actual': {'analysts': 0, 'diplomats': 0, 'explorers': 0, 'sentinels': 0}}
prediction_results['predicted']['sentinels'] = {'actual': {'analysts': 0, 'diplomats': 0, 'explorers': 0, 'sentinels': 0}}

## prediction_result['analysts']['diplomats'] means prediction is analysts but actual value is diplomats

```prediction_results```<span style="font-size:1.1em"> içerisinde tutulan sayaçların değerleri arttırılır.</span>

In [0]:
for i in range(len(predictions)):
  predicted_value = predictions[i]
  actual_value = test_typeClass[i]
  prediction_results['predicted'][predicted_value]['actual'][actual_value] += 1

<span style="font-size:1.1em">JSON formatına çevrilir </span>```dict``` <span style="font-size:1.1em">tipi. Bu sayede daha okunaklı bir şekilde print edilmiş olur. </span>

In [112]:
import json

print(json.dumps(prediction_results, indent = 2))

{
  "predicted": {
    "analysts": {
      "actual": {
        "analysts": 29375,
        "diplomats": 21923,
        "explorers": 7800,
        "sentinels": 13750
      }
    },
    "diplomats": {
      "actual": {
        "analysts": 8644,
        "diplomats": 15377,
        "explorers": 3150,
        "sentinels": 4624
      }
    },
    "explorers": {
      "actual": {
        "analysts": 0,
        "diplomats": 1,
        "explorers": 2,
        "sentinels": 0
      }
    },
    "sentinels": {
      "actual": {
        "analysts": 85,
        "diplomats": 76,
        "explorers": 25,
        "sentinels": 136
      }
    }
  }
}


<span style="font-size:1.1em;">İlgili field extract edilir</span> ```dict``` <span style="font-size:1.1em;">yapısından.</span>

In [0]:
results = prediction_results['predicted']

<span style="font-size:1.1em;">Başarı oranı hesaplanır</span>

In [114]:
accuracy = (results['analysts']['actual']['analysts'] + results['diplomats']['actual']['diplomats'] + results['explorers']['actual']['explorers'] + results['sentinels']['actual']['sentinels']) / len(predictions)
accuracy

0.4276541422147702

<span style="font-size:1.1em;">**E/I** boyutu tahmin edilir</span>

In [0]:
X_train, X_test, y_train, y_test = train_test_split(df['entry'], df['I/E'], random_state = 42)  ## Geri kalanlar S, T, J

X_train_tfidf = tfidf_vectorizer.transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

clf = MultinomialNB().fit(X_train_tfidf, y_train)
test_typeClass = y_test.values

predictions = clf.predict(X_test_tfidf)

In [116]:
predictions

array(['E', 'I', 'E', ..., 'I', 'E', 'I'], dtype='<U1')

In [117]:
predicted = {}
predicted['I'] = {'actual': {'I': 0, 'E': 0}}
predicted['E'] = {'actual': {'I': 0, 'E': 0}}
predicted

{'E': {'actual': {'E': 0, 'I': 0}}, 'I': {'actual': {'E': 0, 'I': 0}}}

In [0]:
for i in range(len(predictions)):
  predicted[predictions[i]]['actual'][test_typeClass[i]] += 1


In [119]:
predicted

{'E': {'actual': {'E': 28792, 'I': 18805}},
 'I': {'actual': {'E': 37614, 'I': 45999}}}

In [0]:
accuracy = (predicted['E']['actual']['E'] + predicted['I']['actual']['I']) / len(predictions)

In [121]:
accuracy

0.5700099077814191

<span style="font-size:1.1em">**S/N** boyutu tahmin edilir.</span>

In [0]:
X_train, X_test, y_train, y_test = train_test_split(df['entry'], df['S/N'], random_state = 42)  ## Geri kalan boyutlar: T, J

X_train_tfidf = tfidf_vectorizer.transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

clf = MultinomialNB().fit(X_train_tfidf, y_train)
test_typeClass = y_test.values

predictions = clf.predict(X_test_tfidf)

In [123]:
predicted['N'] = {'actual': {'N': 0, 'S': 0}}
predicted['S'] = {'actual': {'N': 0, 'S': 0}}

predicted

{'E': {'actual': {'E': 28792, 'I': 18805}},
 'I': {'actual': {'E': 37614, 'I': 45999}},
 'N': {'actual': {'N': 0, 'S': 0}},
 'S': {'actual': {'N': 0, 'S': 0}}}

In [0]:
for i in range(len(predictions)):
  predicted[predictions[i]]['actual'][test_typeClass[i]] += 1

In [125]:
predicted

{'E': {'actual': {'E': 28792, 'I': 18805}},
 'I': {'actual': {'E': 37614, 'I': 45999}},
 'N': {'actual': {'N': 94317, 'S': 36809}},
 'S': {'actual': {'N': 40, 'S': 44}}}

In [0]:
accuracy = (predicted['N']['actual']['N'] + predicted['S']['actual']['S']) / len(predictions)

In [127]:
accuracy

0.7191601249904733

<span style="font-size:1.1em">**T/F** boyutu tahmin edilir.</span>

In [0]:
X_train, X_test, y_train, y_test = train_test_split(df['entry'], df['T/F'], random_state = 42)  ##  J

X_train_tfidf = tfidf_vectorizer.transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

clf = MultinomialNB().fit(X_train_tfidf, y_train)
test_typeClass = y_test.values

predictions = clf.predict(X_test_tfidf)

In [129]:
predicted['T'] = {'actual': {'T': 0, 'F': 0}}
predicted['F'] = {'actual': {'T': 0, 'F': 0}}

predicted

{'E': {'actual': {'E': 28792, 'I': 18805}},
 'F': {'actual': {'F': 0, 'T': 0}},
 'I': {'actual': {'E': 37614, 'I': 45999}},
 'N': {'actual': {'N': 94317, 'S': 36809}},
 'S': {'actual': {'N': 40, 'S': 44}},
 'T': {'actual': {'F': 0, 'T': 0}}}

In [0]:
for i in range(len(predictions)):
  predicted[predictions[i]]['actual'][test_typeClass[i]] += 1

In [131]:
predicted

{'E': {'actual': {'E': 28792, 'I': 18805}},
 'F': {'actual': {'F': 21647, 'T': 13183}},
 'I': {'actual': {'E': 37614, 'I': 45999}},
 'N': {'actual': {'N': 94317, 'S': 36809}},
 'S': {'actual': {'N': 40, 'S': 44}},
 'T': {'actual': {'F': 40822, 'T': 55558}}}

In [0]:
accuracy = (predicted['F']['actual']['F'] + predicted['T']['actual']['T']) / len(predictions)

In [133]:
accuracy

0.588407895739654

<span style="font-size:1.1em">**J/P** boyutu tahmin edilir.</span>

In [0]:
X_train, X_test, y_train, y_test = train_test_split(df['entry'], df['J/P'], random_state = 42) 

X_train_tfidf = tfidf_vectorizer.transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

clf = MultinomialNB().fit(X_train_tfidf, y_train)
test_typeClass = y_test.values

predictions = clf.predict(X_test_tfidf)

In [135]:
predicted['J'] = {'actual': {'J': 0, 'P': 0}}
predicted['P'] = {'actual': {'J': 0, 'P': 0}}

predicted

{'E': {'actual': {'E': 28792, 'I': 18805}},
 'F': {'actual': {'F': 21647, 'T': 13183}},
 'I': {'actual': {'E': 37614, 'I': 45999}},
 'J': {'actual': {'J': 0, 'P': 0}},
 'N': {'actual': {'N': 94317, 'S': 36809}},
 'P': {'actual': {'J': 0, 'P': 0}},
 'S': {'actual': {'N': 40, 'S': 44}},
 'T': {'actual': {'F': 40822, 'T': 55558}}}

In [0]:
for i in range(len(predictions)):
  predicted[predictions[i]]['actual'][test_typeClass[i]] += 1

In [137]:
predicted

{'E': {'actual': {'E': 28792, 'I': 18805}},
 'F': {'actual': {'F': 21647, 'T': 13183}},
 'I': {'actual': {'E': 37614, 'I': 45999}},
 'J': {'actual': {'J': 32889, 'P': 25928}},
 'N': {'actual': {'N': 94317, 'S': 36809}},
 'P': {'actual': {'J': 31223, 'P': 41170}},
 'S': {'actual': {'N': 40, 'S': 44}},
 'T': {'actual': {'F': 40822, 'T': 55558}}}

In [0]:
accuracy = (predicted['P']['actual']['P'] + predicted['J']['actual']['J']) / len(predictions)

In [139]:
accuracy

0.564431064705434

In [140]:
prediction_results['predicted'].update(predicted) 

prediction_results

{'predicted': {'E': {'actual': {'E': 28792, 'I': 18805}},
  'F': {'actual': {'F': 21647, 'T': 13183}},
  'I': {'actual': {'E': 37614, 'I': 45999}},
  'J': {'actual': {'J': 32889, 'P': 25928}},
  'N': {'actual': {'N': 94317, 'S': 36809}},
  'P': {'actual': {'J': 31223, 'P': 41170}},
  'S': {'actual': {'N': 40, 'S': 44}},
  'T': {'actual': {'F': 40822, 'T': 55558}},
  'analysts': {'actual': {'analysts': 29375,
    'diplomats': 21923,
    'explorers': 7800,
    'sentinels': 13750}},
  'diplomats': {'actual': {'analysts': 8644,
    'diplomats': 15377,
    'explorers': 3150,
    'sentinels': 4624}},
  'explorers': {'actual': {'analysts': 0,
    'diplomats': 1,
    'explorers': 2,
    'sentinels': 0}},
  'sentinels': {'actual': {'analysts': 85,
    'diplomats': 76,
    'explorers': 25,
    'sentinels': 136}}}}